these are the dependencies puppeteer needs

In [1]:
!apt install libatk-bridge2.0-dev libatk1.0-dev libxcomposite1 libxdamage1 libxfixes3 libxrandr2 libxtst6 libgbm1 libdrm2 libudev1 libpango-1.0-0 libpangocairo-1.0-0 libglib2.0-0 libfontconfig1 libfreetype6 libexpat1 libxkbcommon0 libxkbcommon-x11-0 libx11-6 libx11-xcb1 libxcb1 libxext6 libxrender1 libdbus-1-3 libatspi2.0-0

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libfontconfig1 is already the newest version (2.13.1-4.2ubuntu5).
libxcb1 is already the newest version (1.14-3ubuntu3).
libxcb1 set to manually installed.
libxdamage1 is already the newest version (1:1.1.5-2build2).
libxdamage1 set to manually installed.
libxext6 is already the newest version (2:1.3.4-1build1).
libxfixes3 is already the newest version (1:6.0.0-1).
libxfixes3 set to manually installed.
libxkbcommon0 is already the newest version (1.4.0-1).
libxkbcommon0 set to manually installed.
libxrandr2 is already the newest version (2:1.5.2-1build1).
libxrandr2 set to manually installed.
libxrender1 is already the newest version (1:0.9.10-1build4).
libdbus-1-3 is already the newest version (1.12.20-2ubuntu4.1).
libdbus-1-3 set to manually installed.
libdrm2 is already the newest version (2.4.113-2~ubuntu0.22.04.1).
libdrm2 set to manually installed.
libexpat1 is already the newest vers

packages we will use

In [2]:
!pip install playwright
!playwright install chromium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 MB 22.6 MB/s eta 0:00:00
(node:733) [DEP0169] DeprecationWarning: `url.parse()` behavior is not standardized and prone to errors that have security implications. Use the WHATWG URL API instead. CVEs are not issued for `url.parse()` vulnerabilities.
(Use `node --trace-deprecation ...` to show where the warning was created)
164.7 MiB [] 0% 334.7s164.7 MiB [] 0% 22.0s164.7 MiB [] 0% 9.9s164.7 MiB [] 0% 6.1s164.7 MiB [] 1% 3.4s164.7 MiB [] 3% 2.5s164.7 MiB [] 4% 2.4s164.7 MiB [] 4% 2.2s164.7 MiB [] 6% 1.9s164.7 MiB [] 7% 1.8s164.7 MiB [] 8% 1.8s164.7 MiB [] 10% 1.6s164.7 MiB [] 11% 1.6s164.7 MiB [] 12% 1.5s164.7 MiB [] 12% 1.6s164.7 MiB [] 14% 1.5s164.7 MiB [] 15% 1.5s164.7 MiB [] 16% 1.5s164.7 MiB [] 17% 1.4s164.7 MiB [] 18% 1.4s164.7 MiB [] 19% 1.4s164.7 MiB [] 21% 1.3s164.7 MiB [] 21% 1.4s164.7 MiB [] 22% 1.3s164.7 MiB [] 24% 1.2s164.7 MiB [] 26% 1.2s164.7 MiB [] 27% 1.1s164.7 MiB [] 28% 1.1s164.7 MiB [] 30% 1.1s164.7 MiB [] 31% 1.0s

In [3]:
import asyncio
from bs4 import BeautifulSoup
from dataclasses import dataclass
from typing import List
import random
import re
from urllib.parse import urljoin
from playwright.async_api import async_playwright

@dataclass
class Headline:
    string: str
    date: str  # YYYYMMDD format

# Realistic user agents
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.1 Safari/605.1.15",
]

def get_random_user_agent() -> str:
    """Return a random user agent string."""
    return random.choice(USER_AGENTS)

def extract_date_from_url(url: str) -> str:
    """Extract date from edition URL and convert to YYYYMMDD format."""
    match = re.search(r'(\d{4})-(\d{2})-(\d{2})', url)
    if match:
        year, month, day = match.groups()
        return f"{year}{month}{day}"
    return ""

async def scrape_archive_index_playwright(url: str) -> List[str]:
    """Use Playwright to scrape the archive index page (handles Cloudflare)."""
    print(f"Using Playwright to fetch {url}...")

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(user_agent=get_random_user_agent())
        page = await context.new_page()

        edition_urls = []
        try:
            await page.goto(url, wait_until="domcontentloaded")
            await page.wait_for_selector('[data-testid="edition-link"]', timeout=10000)

            # Get page HTML
            html = await page.content()
            soup = BeautifulSoup(html, 'html.parser')

            # Find all edition links
            link_elements = soup.find_all(attrs={"data-testid": "edition-link"})

            for element in link_elements:
                href = element.get('href')
                if href:
                    full_url = urljoin(url, href)
                    edition_urls.append(full_url)

            print(f"Found {len(edition_urls)} edition links")
            return edition_urls

        except Exception as e:
            print(f"Error with Playwright: {e}")
            return []
        finally:
            await browser.close()

async def fetch_page_playwright(url: str, max_retries: int = 3) -> str | None:
    """Fetch a page with Playwright (handles Cloudflare) with retry logic."""
    for attempt in range(max_retries):
        try:
            # Random delay between 100-500ms
            await asyncio.sleep(random.uniform(0.1, 0.5))

            async with async_playwright() as p:
                browser = await p.chromium.launch(headless=True)
                context = await browser.new_context(user_agent=get_random_user_agent())
                page = await context.new_page()

                try:
                    await page.goto(url, wait_until="domcontentloaded", timeout=10000)
                    html = await page.content()
                    await browser.close()
                    return html
                except Exception as e:
                    await browser.close()
                    if attempt < max_retries - 1:
                        print(f"Retry {attempt + 1}/{max_retries} for {url}: {e}")
                        await asyncio.sleep(2 ** attempt)
                    else:
                        print(f"Failed to fetch {url}: {e}")
                    continue
        except Exception as e:
            print(f"Error in Playwright: {e}")
            if attempt < max_retries - 1:
                await asyncio.sleep(2 ** attempt)

    return None

async def scrape_edition(edition_url: str) -> List[Headline]:
    """Scrape headlines from a single edition."""
    html = await fetch_page_playwright(edition_url)
    if not html:
        return []

    soup = BeautifulSoup(html, 'html.parser')
    headlines = []
    date = extract_date_from_url(edition_url)

    if not date:
        print(f"Could not extract date from {edition_url}")
        return []

    # Find all teaser headlines
    teaser_elements = soup.find_all(attrs={"data-testid": "teaser-headline"})

    for element in teaser_elements:
        text = element.get_text(strip=True)
        if text:
            headlines.append(Headline(string=text, date=date))

    print(f"Scraped {len(headlines)} headlines from {edition_url}")
    return headlines

async def scrape_editions(edition_urls: List[str], semaphore: asyncio.Semaphore) -> List[Headline]:
    """Scrape multiple editions with concurrency control."""
    async def scrape_with_semaphore(url: str) -> List[Headline]:
        async with semaphore:
            return await scrape_edition(url)

    tasks = [scrape_with_semaphore(url) for url in edition_urls]
    results = await asyncio.gather(*tasks, return_exceptions=True)

    all_headlines = []
    for result in results:
        if isinstance(result, Exception):
            print(f"Error in scraping task: {result}")
        elif isinstance(result, list):
            all_headlines.extend(result)

    return all_headlines

async def scrape_economist_headlines() -> List[Headline]:
    """Main scraper function."""
    archive_url = "https://www.economist.com/weeklyedition/archive?year=2025"

    # Use Playwright to get edition links
    print("Fetching archive index...")
    edition_urls = await scrape_archive_index_playwright(archive_url)

    if not edition_urls:
        print("No editions found")
        return []

    print(f"Starting to scrape {len(edition_urls)} editions with 3 concurrent requests...")
    semaphore = asyncio.Semaphore(3)
    headlines = await scrape_editions(edition_urls, semaphore)

    print(f"\nTotal headlines scraped: {len(headlines)}")
    return headlines

headlines = await scrape_economist_headlines() # dont care
assert headlines is not None
print("\n=== Sample Headlines ===")
for i, headline in enumerate(headlines[:10]):
  print(f"{i+1}. [{headline.date}] {headline.string}")
assert len(headlines) > 10
print(f"\n... and {len(headlines) - 10} more")

Fetching archive index...
Using Playwright to fetch https://www.economist.com/weeklyedition/archive?year=2025...
Found 51 edition links
Starting to scrape 51 editions with 3 concurrent requests...
Scraped 72 headlines from https://www.economist.com/weeklyedition/2025-12-13
Scraped 73 headlines from https://www.economist.com/weeklyedition/2025-12-20
Scraped 74 headlines from https://www.economist.com/weeklyedition/2025-12-06
Scraped 73 headlines from https://www.economist.com/weeklyedition/2025-11-29
Scraped 69 headlines from https://www.economist.com/weeklyedition/2025-11-22
Scraped 70 headlines from https://www.economist.com/weeklyedition/2025-11-15
Scraped 78 headlines from https://www.economist.com/weeklyedition/2025-11-08
Scraped 75 headlines from https://www.economist.com/weeklyedition/2025-11-01
Scraped 69 headlines from https://www.economist.com/weeklyedition/2025-10-25
Scraped 78 headlines from https://www.economist.com/weeklyedition/2025-10-18
Scraped 68 headlines from https:/

In [4]:
print(headlines)
print(headlines[0].string)

[Headline(string='China proved its strengths in 2025—and Donald Trump helped', date='20251220'), Headline(string='Two months in, the Gaza ceasefire is floundering', date='20251220'), Headline(string='The Economist’s country of the year for 2025', date='20251220'), Headline(string='What Novo Nordisk, OpenAI and Pop Mart have in common', date='20251220'), Headline(string='Your Well Informed guide to surviving Christmas', date='20251220'), Headline(string='How the young can make sense of the news', date='20251220'), Headline(string='Europeans should be worried by America’s new national security strategy', date='20251220'), Headline(string='Will California try to block Hollywood’s next megadeal?', date='20251220'), Headline(string='Toll roads are spreading in America', date='20251220'), Headline(string='More schools in America are adopting a four-day week', date='20251220'), Headline(string='The race for an AI Jesus is on', date='20251220'), Headline(string='A vote against gerrymandering s

In [5]:
import json
import numpy as np
from sentence_transformers import SentenceTransformer
import time

def create_embeddings(headlines, model_name="all-mpnet-base-v2"):
    """
    Create embeddings for all headlines.

    Model choices:
    - all-MiniLM-L6-v2: Fast (384-dim), good for most tasks
    - all-mpnet-base-v2: Better quality (768-dim), slower
    - paraphrase-MiniLM-L6-v2: Good for paraphrasing tasks
    """
    print(f"\nLoading model '{model_name}'all-MiniLM-L6-v2")
    model = SentenceTransformer(model_name)

    # Extract just the text for embedding
    texts = [h.string for h in headlines]

    print(f"Encoding {len(texts)} headlines...")
    start = time.time()
    embeddings = model.encode(texts, show_progress_bar=True, convert_to_numpy=True)
    elapsed = time.time() - start

    print(f"Embeddings created in {elapsed:.2f}s")
    print(f"Embedding shape: {embeddings.shape}")
    print(f"Vector dimension: {embeddings.shape[1]}")

    return embeddings, model

def save_embeddings(headlines, embeddings, output_file="headlines_with_embeddings.json"):
    """Save headlines with their embeddings."""
    print(f"\nSaving embeddings to {output_file}...")

    # Create output data structure
    output_data = []
    for i, (headline, embedding) in enumerate(zip(headlines, embeddings)):
        output_data.append({
            "id": i,
            "text": headline.string,
            "date": headline.date,
            "embedding": embedding.tolist()  # Convert numpy array to list for JSON
        })

    with open(output_file, 'w') as f:
        json.dump(output_data, f)

    print(f"Saved {len(output_data)} headlines with embeddings")
    return output_file

def save_embeddings_binary(headlines, embeddings, output_file="headlines_embeddings.npy"):
    """
    Save embeddings as binary NumPy file (more efficient for large-scale use).
    This creates two files:
    - headlines_embeddings.npy: the embedding vectors
    - headlines_metadata.json: the text and dates
    """
    print(f"\nSaving embeddings as binary NumPy file...")

    # Save embeddings as binary
    np.save(output_file, embeddings)
    print(f"Embeddings saved to {output_file}")

    # Save metadata separately (more efficient than storing in JSON)
    metadata_file = output_file.replace(".npy", "_metadata.json")
    metadata = [{"text": h.string, "date": h.date} for h in headlines]

    with open(metadata_file, 'w') as f:
        json.dump(metadata, f)
    print(f"Metadata saved to {metadata_file}")

    return output_file, metadata_file

def demonstrate_similarity(embeddings, headlines, model):
    """Show example of semantic similarity between headlines."""
    print("\n=== Semantic Similarity Example ===")

    # Calculate similarity between first headline and others
    query_embedding = embeddings[0]
    query_text = headlines[0].string

    print(f"Query headline: '{query_text[:80]}...'")
    print(f"\nMost similar headlines:")

    # Calculate cosine similarity
    similarities = np.dot(embeddings, query_embedding) / (
        np.linalg.norm(embeddings, axis=1) * np.linalg.norm(query_embedding)
    )

    # Get top 5 most similar (excluding the first one which is itself)
    top_indices = np.argsort(similarities)[::-1][1:6]

    for rank, idx in enumerate(top_indices, 1):
        similarity_score = similarities[idx]
        text = headlines[idx].string[:80]
        print(f"{rank}. (similarity: {similarity_score:.3f}) {text}...")

# Create embeddings
embeddings, model = create_embeddings(headlines)

# Save in both formats
json_output = save_embeddings(headlines, embeddings)
npy_output, metadata_output = save_embeddings_binary(headlines, embeddings)

# Show similarity example
demonstrate_similarity(embeddings, headlines, model)

print("\n=== Done! ===")
print(f"JSON format (full data): {json_output}")
print(f"Binary format (efficient): {npy_output}")



Loading model 'all-mpnet-base-v2'all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Encoding 3661 headlines...


Batches:   0%|          | 0/115 [00:00<?, ?it/s]

Embeddings created in 4.30s
Embedding shape: (3661, 768)
Vector dimension: 768

Saving embeddings to headlines_with_embeddings.json...
Saved 3661 headlines with embeddings

Saving embeddings as binary NumPy file...
Embeddings saved to headlines_embeddings.npy
Metadata saved to headlines_embeddings_metadata.json

=== Semantic Similarity Example ===
Query headline: 'China proved its strengths in 2025—and Donald Trump helped...'

Most similar headlines:
1. (similarity: 0.731) How America could end up making China great again...
2. (similarity: 0.725) China’s leaders reveal their plan to cope with 2025...
3. (similarity: 0.701) As Donald Trump’s trade war heats up, China is surprisingly confident...
4. (similarity: 0.668) Donald Trump and Xi Jinping: both weaker than they think...
5. (similarity: 0.661) Xi Jinping has much to worry about in 2025...

=== Done! ===
JSON format (full data): headlines_with_embeddings.json
Binary format (efficient): headlines_embeddings.npy


In [7]:
import json
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from collections import defaultdict
from datetime import datetime

def load_embeddings_and_metadata():
    """Load embeddings and metadata from saved files."""
    print("Loading embeddings and metadata...")
    embeddings = np.load("headlines_embeddings.npy")

    with open("headlines_embeddings_metadata.json", 'r') as f:
        metadata = json.load(f)

    print(f"Loaded {len(embeddings)} embeddings")
    print(f"Embedding dimension: {embeddings.shape[1]}")
    return embeddings, metadata

def cluster_embeddings(embeddings, n_clusters=12):
    """Cluster embeddings using Gaussian mixture models."""
    print(f"\nClustering embeddings into {n_clusters} clusters...")
    gmm = GaussianMixture(
        n_components=n_clusters,
        random_state=42,
        n_init=10,
        max_iter=200,
        verbose=1
    )
    cluster_labels = gmm.fit_predict(embeddings)

    # Calculate silhouette score
    silhouette_avg = silhouette_score(embeddings, cluster_labels)
    print(f"Silhouette Score: {silhouette_avg:.3f}")

    return gmm, cluster_labels

def group_by_date_and_cluster(metadata, cluster_labels):
    """Group headlines by date and cluster."""
    print("\nGrouping by date and cluster...")

    # Dictionary: date -> set of cluster IDs present on that date
    date_clusters = defaultdict(set)
    # Dictionary: (date, cluster_id) -> list of headlines
    date_cluster_headlines = defaultdict(list)

    for idx, (item, cluster_id) in enumerate(zip(metadata, cluster_labels)):
        date = item['date']
        text = item['text']

        date_clusters[date].add(cluster_id)
        date_cluster_headlines[(date, cluster_id)].append(text)

    # Sort dates
    sorted_dates = sorted(date_clusters.keys())

    print(f"Found {len(sorted_dates)} unique dates (magazine issues)")
    print(f"Date range: {sorted_dates[0]} to {sorted_dates[-1]}")

    return sorted_dates, date_clusters, date_cluster_headlines

def find_disappeared_topics(sorted_dates, date_clusters, date_cluster_headlines):
    """Find topics (clusters) that disappeared between consecutive issues."""
    print("\n=== TOPIC DECAY ANALYSIS ===")
    print("Topics present in issue N-2 but NOT in issue N-1")
    print("=" * 80)

    results = []

    for i in range(1, len(sorted_dates)):
        older_date = sorted_dates[i - 1]  # N-2 (earlier issue)
        newer_date = sorted_dates[i]       # N-1 (later issue)

        # Find clusters that disappeared
        disappeared_clusters = date_clusters[older_date] - date_clusters[newer_date]

        if disappeared_clusters:
            print(f"\n📉 Between {older_date} → {newer_date}")
            print(f"   {len(disappeared_clusters)} topic(s) disappeared:")

            for cluster_id in sorted(disappeared_clusters):
                headlines = date_cluster_headlines[(older_date, cluster_id)]
                print(f"\n   Cluster {cluster_id} (had {len(headlines)} headlines in {older_date}):")

                # Show up to 3 representative headlines from this cluster
                for j, headline in enumerate(headlines[:3], 1):
                    truncated = headline[:70] + "..." if len(headline) > 70 else headline
                    print(f"     • {truncated}")

                if len(headlines) > 3:
                    print(f"     ... and {len(headlines) - 3} more")

                results.append({
                    'older_date': str(older_date),
                    'newer_date':  str(newer_date),
                    'cluster_id': str(cluster_id),
                    'headline_count': str(len(headlines)),
                    'sample_headlines': headlines[:3]
                })

    return results

def analyze_cluster_persistence(sorted_dates, date_clusters):
    """Analyze which clusters are most persistent across issues."""
    print("\n=== CLUSTER PERSISTENCE ANALYSIS ===")
    print("=" * 80)

    cluster_persistence = defaultdict(int)
    total_dates = len(sorted_dates)

    for date in sorted_dates:
        for cluster_id in date_clusters[date]:
            cluster_persistence[cluster_id] += 1

    # Sort by persistence
    sorted_persistence = sorted(cluster_persistence.items(), key=lambda x: x[1], reverse=True)

    print(f"\nCluster persistence (% of issues where cluster appears):")
    for cluster_id, count in sorted_persistence:
        percentage = (count / total_dates) * 100
        bar = "█" * (int(percentage / 5)) + "░" * (20 - int(percentage / 5))
        print(f"Cluster {cluster_id:2d}: {bar} {percentage:5.1f}% ({count}/{total_dates})")

def save_results(results, output_file="topic_decay_results.json"):
    """Save decay analysis results to JSON."""
    print(f"\nSaving results to {output_file}...")
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=2)
    print(f"Saved {len(results)} decay events")

# Load data
embeddings, metadata = load_embeddings_and_metadata()

# Cluster embeddings
gmm, cluster_labels = cluster_embeddings(embeddings, n_clusters=20)

# Group by date and cluster
sorted_dates, date_clusters, date_cluster_headlines = group_by_date_and_cluster(
    metadata, cluster_labels
)

# Find disappeared topics
decay_results = find_disappeared_topics(sorted_dates, date_clusters, date_cluster_headlines)

# Analyze cluster persistence
analyze_cluster_persistence(sorted_dates, date_clusters)

# Save results
save_results(decay_results)

Loading embeddings and metadata...
Loaded 3661 embeddings
Embedding dimension: 768

Clustering embeddings into 20 clusters...
Initialization 0
Initialization converged.
Initialization 1
Initialization converged.
Initialization 2
Initialization converged.
Initialization 3
Initialization converged.
Initialization 4
Initialization converged.
Initialization 5
Initialization converged.
Initialization 6
Initialization converged.
Initialization 7
Initialization converged.
Initialization 8
Initialization converged.
Initialization 9
Initialization converged.
Silhouette Score: 0.044

Grouping by date and cluster...
Found 51 unique dates (magazine issues)
Date range: 20250104 to 20251220

=== TOPIC DECAY ANALYSIS ===
Topics present in issue N-2 but NOT in issue N-1

📉 Between 20250111 → 20250118
   1 topic(s) disappeared:

   Cluster 9 (had 1 headlines in 20250111):
     • Training AI models might not need enormous data centres

📉 Between 20250201 → 20250208
   2 topic(s) disappeared:

   Cluster

In [8]:
import json
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from collections import defaultdict, Counter

def load_data():
    """Load embeddings and metadata."""
    embeddings = np.load("headlines_embeddings.npy")
    with open("headlines_embeddings_metadata.json", 'r') as f:
        metadata = json.load(f)
    return embeddings, metadata

def evaluate_cluster_quality(embeddings, cluster_labels):
    """Calculate cluster quality metrics."""
    print("=" * 80)
    print("CLUSTER QUALITY METRICS")
    print("=" * 80)

    silhouette = silhouette_score(embeddings, cluster_labels)
    davies_bouldin = davies_bouldin_score(embeddings, cluster_labels)
    calinski_harabasz = calinski_harabasz_score(embeddings, cluster_labels)

    print(f"\nSilhouette Score: {silhouette:.3f}")
    print(f"  → Range: [-1, 1]. Closer to 1 is better.")
    print(f"  → {silhouette:.3f}: {'✓ Good' if silhouette > 0.5 else '⚠ Moderate' if silhouette > 0.3 else '✗ Poor'}")

    print(f"\nDavies-Bouldin Index: {davies_bouldin:.3f}")
    print(f"  → Lower is better. Measures cluster separation.")
    print(f"  → {davies_bouldin:.3f}: {'✓ Good' if davies_bouldin < 1.0 else '⚠ Moderate' if davies_bouldin < 1.5 else '✗ Poor'}")

    print(f"\nCalinski-Harabasz Score: {calinski_harabasz:.1f}")
    print(f"  → Higher is better. Ratio of between-cluster to within-cluster variance.")
    print(f"  → {calinski_harabasz:.1f}: {'✓ Good' if calinski_harabasz > 50 else '⚠ Moderate' if calinski_harabasz > 20 else '✗ Poor'}")

    return silhouette, davies_bouldin, calinski_harabasz

def evaluate_gmm_fit(gmm, embeddings):
    """Evaluate GMM-specific metrics."""
    print("\n" + "=" * 80)
    print("GAUSSIAN MIXTURE MODEL METRICS")
    print("=" * 80)

    bic = gmm.bic(embeddings)
    aic = gmm.aic(embeddings)
    log_likelihood = gmm.score(embeddings)

    print(f"\nBIC (Bayesian Information Criterion): {bic:.2f}")
    print(f"  → Lower is better. Penalizes model complexity.")

    print(f"\nAIC (Akaike Information Criterion): {aic:.2f}")
    print(f"  → Lower is better. Balances fit and complexity.")

    print(f"\nLog-Likelihood: {log_likelihood:.4f}")
    print(f"  → Higher is better. Measures how well model fits data.")

    print(f"\nConverged: {gmm.converged_}")
    print(f"Iterations used: {gmm.n_iter_}")

    return bic, aic, log_likelihood

def evaluate_semantic_coherence(metadata, cluster_labels):
    """Show sample headlines from each cluster to assess semantic coherence."""
    print("\n" + "=" * 80)
    print("SEMANTIC COHERENCE - Sample Headlines by Cluster")
    print("=" * 80)
    print("(Review if headlines within clusters are semantically related)\n")

    # Group headlines by cluster
    cluster_headlines = defaultdict(list)
    for headline, cluster_id in zip(metadata, cluster_labels):
        cluster_headlines[cluster_id].append(headline['text'])

    for cluster_id in sorted(cluster_headlines.keys()):
        headlines = cluster_headlines[cluster_id]
        print(f"Cluster {cluster_id} ({len(headlines)} headlines):")

        # Show up to 4 sample headlines
        for i, headline in enumerate(headlines[:4], 1):
            truncated = headline[:75] + "..." if len(headline) > 75 else headline
            print(f"  {i}. {truncated}")

        if len(headlines) > 4:
            print(f"  ... and {len(headlines) - 4} more")
        print()

def evaluate_coverage(metadata, cluster_labels):
    """Analyze which clusters capture most headlines."""
    print("=" * 80)
    print("COVERAGE ANALYSIS")
    print("=" * 80)

    cluster_counts = Counter(cluster_labels)
    total_headlines = len(cluster_labels)

    print(f"\nTotal headlines: {total_headlines}")
    print(f"Number of clusters: {len(cluster_counts)}")
    print(f"\nCluster coverage (largest to smallest):\n")

    for cluster_id, count in cluster_counts.most_common():
        percentage = (count / total_headlines) * 100
        bar = "█" * (int(percentage / 2)) + "░" * (50 - int(percentage / 2))
        print(f"Cluster {cluster_id:2d}: {bar} {percentage:5.1f}% ({count:4d} headlines)")

def evaluate_decay_validity(metadata, cluster_labels):
    """Analyze decay events for validity and patterns."""
    print("\n" + "=" * 80)
    print("DECAY EVENT VALIDITY")
    print("=" * 80)

    # Group by date
    date_clusters = defaultdict(set)
    date_cluster_headlines = defaultdict(list)

    for idx, (item, cluster_id) in enumerate(zip(metadata, cluster_labels)):
        date = item['date']
        text = item['text']
        date_clusters[date].add(cluster_id)
        date_cluster_headlines[(date, cluster_id)].append(text)

    sorted_dates = sorted(date_clusters.keys())

    # Analyze decay patterns
    cluster_decay_count = Counter()
    for i in range(1, len(sorted_dates)):
        older_date = sorted_dates[i - 1]
        newer_date = sorted_dates[i]
        disappeared = date_clusters[older_date] - date_clusters[newer_date]
        for cluster_id in disappeared:
            cluster_decay_count[cluster_id] += 1

    print(f"\nTotal decay events: {sum(cluster_decay_count.values())}")
    print(f"Unique clusters that decayed: {len(cluster_decay_count)}")
    print(f"\nClusters with most decay events (volatility):\n")

    for cluster_id, count in cluster_decay_count.most_common(10):
        print(f"Cluster {cluster_id:2d}: {count} decay events (appears/disappears frequently)")

    # Check if decay is correlated with cluster size
    print(f"\n\nDecay correlation with cluster size:")
    print(f"Do smaller clusters decay more often?\n")

    cluster_sizes = Counter(cluster_labels)
    decay_rates = {}

    for cluster_id in cluster_sizes:
        size = cluster_sizes[cluster_id]
        decays = cluster_decay_count[cluster_id]
        decay_rate = (decays / len(sorted_dates)) if len(sorted_dates) > 0 else 0
        decay_rates[cluster_id] = (size, decays, decay_rate)

    # Sort by cluster size
    for cluster_id, (size, decays, rate) in sorted(decay_rates.items(), key=lambda x: x[1][0], reverse=True):
        bar = "█" * (int(rate * 20)) + "░" * (20 - int(rate * 20))
        print(f"Cluster {cluster_id:2d} (size: {size:4d}): {bar} {rate*100:5.1f}% decay rate")

def evaluate_temporal_patterns(metadata, cluster_labels):
    """Identify if clusters correspond to real-world events."""
    print("\n" + "=" * 80)
    print("TEMPORAL PATTERNS - Topics Over Time")
    print("=" * 80)

    date_clusters = defaultdict(set)
    date_cluster_headlines = defaultdict(list)

    for idx, (item, cluster_id) in enumerate(zip(metadata, cluster_labels)):
        date = item['date']
        text = item['text']
        date_clusters[date].add(cluster_id)
        date_cluster_headlines[(date, cluster_id)].append(text)

    sorted_dates = sorted(date_clusters.keys())

    print(f"\nDate range: {sorted_dates[0]} to {sorted_dates[-1]}")
    print(f"Number of issues: {len(sorted_dates)}\n")

    # Show which clusters are trending (appear in more recent issues)
    first_half = sorted_dates[:len(sorted_dates)//2]
    second_half = sorted_dates[len(sorted_dates)//2:]

    first_half_clusters = set()
    second_half_clusters = set()

    for date in first_half:
        first_half_clusters.update(date_clusters[date])
    for date in second_half:
        second_half_clusters.update(date_clusters[date])

    emerging = second_half_clusters - first_half_clusters
    fading = first_half_clusters - second_half_clusters
    persistent = first_half_clusters & second_half_clusters

    print(f"Emerging clusters (only in 2nd half): {sorted(emerging) if emerging else 'None'}")
    print(f"Fading clusters (only in 1st half): {sorted(fading) if fading else 'None'}")
    print(f"Persistent clusters (both halves): {len(persistent)} clusters")

    if emerging:
        print(f"\nSample headlines from emerging clusters:")
        for cluster_id in sorted(emerging)[:2]:
            sample = date_cluster_headlines[(sorted_dates[-1], cluster_id)][0]
            print(f"  Cluster {cluster_id}: {sample[:70]}...")

embeddings, metadata = load_data()

# Cluster using Gaussian Mixture Model
print("Clustering embeddings with Gaussian Mixture Model...")
gmm = GaussianMixture(
    n_components=20,
    random_state=42,
    n_init=10,
    max_iter=200,
    verbose=1
)
cluster_labels = gmm.fit_predict(embeddings)

# Run evaluations
evaluate_cluster_quality(embeddings, cluster_labels)
evaluate_gmm_fit(gmm, embeddings)
evaluate_semantic_coherence(metadata, cluster_labels)
evaluate_coverage(metadata, cluster_labels)
evaluate_decay_validity(metadata, cluster_labels)
evaluate_temporal_patterns(metadata, cluster_labels)

print("\n" + "=" * 80)
print("EVALUATION COMPLETE")
print("=" * 80)

Clustering embeddings with Gaussian Mixture Model...
Initialization 0
Initialization converged.
Initialization 1
Initialization converged.
Initialization 2
Initialization converged.
Initialization 3
Initialization converged.
Initialization 4
Initialization converged.
Initialization 5
Initialization converged.
Initialization 6
Initialization converged.
Initialization 7
Initialization converged.
Initialization 8
Initialization converged.
Initialization 9
Initialization converged.
CLUSTER QUALITY METRICS

Silhouette Score: 0.044
  → Range: [-1, 1]. Closer to 1 is better.
  → 0.044: ✗ Poor

Davies-Bouldin Index: 4.097
  → Lower is better. Measures cluster separation.
  → 4.097: ✗ Poor

Calinski-Harabasz Score: 41.7
  → Higher is better. Ratio of between-cluster to within-cluster variance.
  → 41.7: ⚠ Moderate

GAUSSIAN MIXTURE MODEL METRICS

BIC (Bayesian Information Criterion): 20917945.97
  → Lower is better. Penalizes model complexity.

AIC (Akaike Information Criterion): -15826625.31
 

Bad eval results + decent clustering -> eval not descriptive.

Clustering embeddings with Gaussian Mixture Model...
Initialization 0
Initialization converged.
Initialization 1
Initialization converged.
Initialization 2
Initialization converged.
Initialization 3
Initialization converged.
Initialization 4
Initialization converged.
Initialization 5
Initialization converged.
Initialization 6
Initialization converged.
Initialization 7
Initialization converged.
Initialization 8
Initialization converged.
Initialization 9
Initialization converged.
================================================================================
CLUSTER QUALITY METRICS
================================================================================

Silhouette Score: 0.044
  → Range: [-1, 1]. Closer to 1 is better.
  → 0.044: ✗ Poor

Davies-Bouldin Index: 4.097
  → Lower is better. Measures cluster separation.
  → 4.097: ✗ Poor

Calinski-Harabasz Score: 41.7
  → Higher is better. Ratio of between-cluster to within-cluster variance.
  → 41.7: ⚠ Moderate

================================================================================
GAUSSIAN MIXTURE MODEL METRICS
================================================================================

BIC (Bayesian Information Criterion): 20917945.97
  → Lower is better. Penalizes model complexity.

AIC (Akaike Information Criterion): -15826625.31
  → Lower is better. Balances fit and complexity.

Log-Likelihood: 3778.9160
  → Higher is better. Measures how well model fits data.

Converged: True
Iterations used: 2

================================================================================
SEMANTIC COHERENCE - Sample Headlines by Cluster
================================================================================
(Review if headlines within clusters are semantically related)

Cluster 0 (198 headlines):
  1. Two months in, the Gaza ceasefire is floundering
  2. Can Australia defend itself against jihadist murderers?
  3. Donald Trump’s peace plan is faltering in the chaos of Gaza
  4. Just 74 intensive-care beds remain in Gaza
  ... and 194 more

Cluster 1 (279 headlines):
  1. How to heal the trauma from Northern Ireland’s killings
  2. Britons are becoming obsessed with pet photography
  3. A portrait of Britain’s aristocrats
  4. What street talk reveals about Anglophone civilisation
  ... and 275 more

Cluster 2 (205 headlines):
  1. Europeans should be worried by America’s new national security strategy
  2. Italy is using the Winter Olympics to appeal to the ultra-wealthy
  3. Why German cities feel like war zones on New Year’s Eve
  4. European nationalism is dead. Long live European gastronationalism
  ... and 201 more

Cluster 3 (110 headlines):
  1. From honeycomb curry to blood fry: India’s “untouchable” cooking
  2. How did one airline bring Indian aviation to its knees?
  3. India’s best-known artist gets his own museum—in Qatar
  4. India’s defence-tech startups are thriving
  ... and 106 more

Cluster 4 (234 headlines):
  1. Your Well Informed guide to surviving Christmas
  2. How the young can make sense of the news
  3. What The Economist discovered at dating bootcamp
  4. The gigantic task of catering on the world’s biggest cruise ship
  ... and 230 more

Cluster 5 (51 headlines):
  1. Economic data, commodities and markets
  2. Economic data, commodities and markets
  3. Economic data, commodities and markets
  4. Economic data, commodities and markets
  ... and 47 more

Cluster 6 (286 headlines):
  1. The tariffs that nearly stole Christmas
  2. America’s Supreme Court should strike down Donald Trump’s tariffs
  3. The Supreme Court is handing Donald Trump more power
  4. Donald Trump’s flawed plan to get China hooked on Nvidia chips
  ... and 282 more

Cluster 7 (173 headlines):
  1. Ukraine scrabbles for handholds against Russia’s massive assault
  2. Two centuries ago, Russian revolutionaries tried to change the world
  3. Russia is not as resilient as it wants you to think
  4. Europe bans Russia’s gas exports, but still buys its gas-based fertiliser
  ... and 169 more

Cluster 8 (181 headlines):
  1. How humankind’s 10m-year love affair with booze might end
  2. How dogs make teens feel less anxious
  3. Are some types of sugar healthier than others?
  4. What goes into raising the turkey on your holiday table
  ... and 177 more

Cluster 9 (151 headlines):
  1. The race for an AI Jesus is on
  2. How magicians stay relevant in the age of AI
  3. Ditch textbooks and learn how to use a wrench to AI-proof your job?
  4. Job apocalypse? Not yet. AI is creating brand new occupations
  ... and 147 more

Cluster 10 (132 headlines):
  1. Ethiopia wants to build Africa’s biggest airport
  2. Meet the American investors rushing into Congo
  3. A giant iron-ore mine could bring Guinea riches or ruin
  4. Donald Trump has not ended conflict between Congo and Rwanda
  ... and 128 more

Cluster 11 (243 headlines):
  1. Jimmy Lai’s judgment day
  2. All sides have learned a lot from Extinction Rebellion’s co-founder
  3. Adam Smith is misinterpreted and his influence overstated
  4. The long, strange journey of a temple from profane to sacred
  ... and 239 more

Cluster 12 (206 headlines):
  1. A vote against gerrymandering shows why political courage is rare
  2. More reasons for America’s friends to plan for the worst
  3. How much does America know about its boat-strike targets?
  4. All hail “The President of Peace”
  ... and 202 more

Cluster 13 (203 headlines):
  1. The Christmas-industrial complex centres on Yiwu
  2. A journey along East Asia’s hidden artery
  3. Why many Asian megacities are miserable places
  4. The meaning of China’s record-high trade surplus
  ... and 199 more

Cluster 14 (172 headlines):
  1. The battle to stop clever people betting
  2. Where America’s most prominent short-sellers are placing their bets
  3. This Christmas, raise a glass to concentrated market returns
  4. Crypto’s real threat to banks
  ... and 168 more

Cluster 15 (177 headlines):
  1. The Economist’s country of the year for 2025
  2. Javier Milei loosens his grip on the peso
  3. The botched response to a devastating storm infuriates Indonesians
  4. Miami elects a new mayor at a pivotal moment
  ... and 173 more

Cluster 16 (135 headlines):
  1. Will California try to block Hollywood’s next megadeal?
  2. An oil boom where the Amazon meets the Atlantic
  3. A journey into the world’s most mysterious rainforest
  4. Retreating from EVs could be hazardous for Western carmakers
  ... and 131 more

Cluster 17 (162 headlines):
  1. China proved its strengths in 2025—and Donald Trump helped
  2. Why has the border between the Koreas fallen silent?
  3. America’s fight back against China starts in Los Angeles—in flip-flops
  4. Don’t fear China’s trillion-dollar trade surplus
  ... and 158 more

Cluster 18 (198 headlines):
  1. What Novo Nordisk, OpenAI and Pop Mart have in common
  2. The plan to rescue Novo Nordisk
  3. Luxury handbags may be shoddier than you think
  4. SpaceX, OpenAI, Anthropic and their giga-IPO dreams
  ... and 194 more

Cluster 19 (165 headlines):
  1. Toll roads are spreading in America
  2. More schools in America are adopting a four-day week
  3. How to survive abandonment by America
  4. Watch who you’re calling childless
  ... and 161 more

================================================================================
COVERAGE ANALYSIS
================================================================================

Total headlines: 3661
Number of clusters: 20

Cluster coverage (largest to smallest):

Cluster  6: ███░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   7.8% ( 286 headlines)
Cluster  1: ███░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   7.6% ( 279 headlines)
Cluster 11: ███░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   6.6% ( 243 headlines)
Cluster  4: ███░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   6.4% ( 234 headlines)
Cluster 12: ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   5.6% ( 206 headlines)
Cluster  2: ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   5.6% ( 205 headlines)
Cluster 13: ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   5.5% ( 203 headlines)
Cluster  0: ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   5.4% ( 198 headlines)
Cluster 18: ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   5.4% ( 198 headlines)
Cluster  8: ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   4.9% ( 181 headlines)
Cluster 15: ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   4.8% ( 177 headlines)
Cluster  7: ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   4.7% ( 173 headlines)
Cluster 14: ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   4.7% ( 172 headlines)
Cluster 19: ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   4.5% ( 165 headlines)
Cluster 17: ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   4.4% ( 162 headlines)
Cluster  9: ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   4.1% ( 151 headlines)
Cluster 16: █░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   3.7% ( 135 headlines)
Cluster 10: █░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   3.6% ( 132 headlines)
Cluster  3: █░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   3.0% ( 110 headlines)
Cluster  5: ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   1.4% (  51 headlines)

================================================================================
DECAY EVENT VALIDITY
================================================================================

Total decay events: 20
Unique clusters that decayed: 11

Clusters with most decay events (volatility):

Cluster  9: 4 decay events (appears/disappears frequently)
Cluster  3: 4 decay events (appears/disappears frequently)
Cluster 17: 2 decay events (appears/disappears frequently)
Cluster 16: 2 decay events (appears/disappears frequently)
Cluster 18: 2 decay events (appears/disappears frequently)
Cluster 13: 1 decay events (appears/disappears frequently)
Cluster 14: 1 decay events (appears/disappears frequently)
Cluster 15: 1 decay events (appears/disappears frequently)
Cluster  2: 1 decay events (appears/disappears frequently)
Cluster  4: 1 decay events (appears/disappears frequently)


Decay correlation with cluster size:
Do smaller clusters decay more often?

Cluster  6 (size:  286): ░░░░░░░░░░░░░░░░░░░░   2.0% decay rate
Cluster  1 (size:  279): ░░░░░░░░░░░░░░░░░░░░   0.0% decay rate
Cluster 11 (size:  243): ░░░░░░░░░░░░░░░░░░░░   0.0% decay rate
Cluster  4 (size:  234): ░░░░░░░░░░░░░░░░░░░░   2.0% decay rate
Cluster 12 (size:  206): ░░░░░░░░░░░░░░░░░░░░   0.0% decay rate
Cluster  2 (size:  205): ░░░░░░░░░░░░░░░░░░░░   2.0% decay rate
Cluster 13 (size:  203): ░░░░░░░░░░░░░░░░░░░░   2.0% decay rate
Cluster  0 (size:  198): ░░░░░░░░░░░░░░░░░░░░   0.0% decay rate
Cluster 18 (size:  198): ░░░░░░░░░░░░░░░░░░░░   3.9% decay rate
Cluster  8 (size:  181): ░░░░░░░░░░░░░░░░░░░░   0.0% decay rate
Cluster 15 (size:  177): ░░░░░░░░░░░░░░░░░░░░   2.0% decay rate
Cluster  7 (size:  173): ░░░░░░░░░░░░░░░░░░░░   0.0% decay rate
Cluster 14 (size:  172): ░░░░░░░░░░░░░░░░░░░░   2.0% decay rate
Cluster 19 (size:  165): ░░░░░░░░░░░░░░░░░░░░   0.0% decay rate
Cluster 17 (size:  162): ░░░░░░░░░░░░░░░░░░░░   3.9% decay rate
Cluster  9 (size:  151): █░░░░░░░░░░░░░░░░░░░   7.8% decay rate
Cluster 16 (size:  135): ░░░░░░░░░░░░░░░░░░░░   3.9% decay rate
Cluster 10 (size:  132): ░░░░░░░░░░░░░░░░░░░░   0.0% decay rate
Cluster  3 (size:  110): █░░░░░░░░░░░░░░░░░░░   7.8% decay rate
Cluster  5 (size:   51): ░░░░░░░░░░░░░░░░░░░░   0.0% decay rate

CLUSTER QUALITY METRICS
================================================================================

Silhouette Score: 0.045
  → Range: [-1, 1]. Closer to 1 is better.
  → 0.045: ✗ Poor

Davies-Bouldin Index: 3.966
  → Lower is better. Measures cluster separation.
  → 3.966: ✗ Poor

Calinski-Harabasz Score: 40.4
  → Higher is better. Ratio of between-cluster to within-cluster variance.
  → 40.4: ⚠ Moderate

Clustering embeddings...
================================================================================
CLUSTER QUALITY METRICS
================================================================================

Silhouette Score: 0.044
  → Range: [-1, 1]. Closer to 1 is better.
  → 0.044: ✗ Poor

Davies-Bouldin Index: 3.950
  → Lower is better. Measures cluster separation.
  → 3.950: ✗ Poor

Calinski-Harabasz Score: 41.9
  → Higher is better. Ratio of between-cluster to within-cluster variance.
  → 41.9: ⚠ Moderate


Similar results with better embedding model mpnet..
Try gaussian mixture models

In [9]:
import json
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict, Counter
import re

def load_data():
    embeddings = np.load("headlines_embeddings.npy")
    with open("headlines_embeddings_metadata.json", 'r') as f:
        metadata = json.load(f)
    return embeddings, metadata

def cluster_data(embeddings):
    """Cluster using GMM."""
    gmm = GaussianMixture(n_components=20, random_state=42, n_init=10)
    return gmm.fit_predict(embeddings)

def normalize_embeddings(embeddings):
    """Normalize embeddings for cosine similarity."""
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return embeddings / (norms + 1e-10)

def compute_semantic_homogeneity(embeddings, cluster_labels):
    """Measure avg similarity of headlines WITHIN clusters."""
    print("\n" + "="*80)
    print("1. SEMANTIC HOMOGENEITY (within-cluster similarity)")
    print("="*80)
    print("   Higher = headlines within cluster are similar to each other")

    normalized = normalize_embeddings(embeddings)
    cluster_homogeneity = {}

    for cluster_id in np.unique(cluster_labels):
        cluster_mask = cluster_labels == cluster_id
        cluster_embeddings = normalized[cluster_mask]

        if len(cluster_embeddings) < 2:
            continue

        # Compute pairwise similarities
        similarities = np.dot(cluster_embeddings, cluster_embeddings.T)
        # Get upper triangle (avoid duplicates and diagonal)
        n = len(cluster_embeddings)
        similarities_flat = similarities[np.triu_indices(n, k=1)]

        avg_similarity = similarities_flat.mean()
        cluster_homogeneity[cluster_id] = avg_similarity

    # Print results
    sorted_clusters = sorted(cluster_homogeneity.items(), key=lambda x: x[1], reverse=True)
    print(f"\n   Cluster homogeneity scores:")
    for cluster_id, score in sorted_clusters:
        bar = "█" * int(score * 20) + "░" * (20 - int(score * 20))
        print(f"   Cluster {cluster_id:2d}: {bar} {score:.3f}")

    avg_homogeneity = np.mean(list(cluster_homogeneity.values()))
    print(f"\n   Overall avg homogeneity: {avg_homogeneity:.3f}")
    print(f"   → {avg_homogeneity:.3f}: {'✓ Good' if avg_homogeneity > 0.6 else '⚠ Moderate' if avg_homogeneity > 0.4 else '✗ Needs improvement'}")

    return cluster_homogeneity

def compute_semantic_separation(embeddings, cluster_labels):
    """Measure similarity of headlines ACROSS clusters (should be low)."""
    print("\n" + "="*80)
    print("2. SEMANTIC SEPARATION (between-cluster dissimilarity)")
    print("="*80)
    print("   Lower = clusters are distinct from each other")

    normalized = normalize_embeddings(embeddings)
    unique_clusters = np.unique(cluster_labels)

    between_cluster_similarities = []

    for i, c1 in enumerate(unique_clusters):
        for c2 in unique_clusters[i+1:]:
            cluster1_mask = cluster_labels == c1
            cluster2_mask = cluster_labels == c2
            cluster1_embeddings = normalized[cluster1_mask]
            cluster2_embeddings = normalized[cluster2_mask]

            # Random sample to avoid too many comparisons
            sample_size = min(20, len(cluster1_embeddings), len(cluster2_embeddings))
            idx1 = np.random.choice(len(cluster1_embeddings), sample_size, replace=False)
            idx2 = np.random.choice(len(cluster2_embeddings), sample_size, replace=False)

            similarities = np.dot(cluster1_embeddings[idx1], cluster2_embeddings[idx2].T)
            between_cluster_similarities.extend(similarities.flatten())

    between_cluster_similarities = np.array(between_cluster_similarities)
    avg_separation = between_cluster_similarities.mean()

    print(f"\n   Between-cluster similarity: {avg_separation:.3f}")
    print(f"   (Headlines from different clusters have {avg_separation:.3f} avg similarity)")
    print(f"   → {avg_separation:.3f}: {'✓ Good separation' if avg_separation < 0.5 else '⚠ Moderate' if avg_separation < 0.6 else '✗ Poor separation'}")

    return avg_separation

def extract_distinctive_words(texts, n_words=5):
    """Extract distinctive words from a group of texts using TF-IDF."""
    if not texts or len(texts) < 2:
        return []

    vectorizer = TfidfVectorizer(max_features=100, stop_words='english', min_df=1)
    try:
        tfidf_matrix = vectorizer.fit_transform(texts)
        # Get average TF-IDF score for each word
        avg_tfidf = np.asarray(tfidf_matrix.mean(axis=0)).flatten()
        top_indices = np.argsort(avg_tfidf)[::-1][:n_words]
        words = [vectorizer.get_feature_names_out()[i] for i in top_indices]
        return words
    except:
        return []

def compute_topic_distinctiveness(metadata, cluster_labels):
    """Measure how distinct each cluster's vocabulary is."""
    print("\n" + "="*80)
    print("3. TOPIC DISTINCTIVENESS (unique vocabulary per cluster)")
    print("="*80)
    print("   Shows if clusters have unique, interpretable topics")

    cluster_texts = defaultdict(list)
    for headline, cluster_id in zip(metadata, cluster_labels):
        cluster_texts[cluster_id].append(headline['text'])

    print(f"\n   Top distinctive words per cluster:\n")

    for cluster_id in sorted(cluster_texts.keys()):
        words = extract_distinctive_words(cluster_texts[cluster_id], n_words=5)
        if words:
            print(f"   Cluster {cluster_id:2d}: {', '.join(words)}")

    # Check word overlap between clusters (should be low)
    all_cluster_words = {}
    for cluster_id in cluster_texts.keys():
        words = set(extract_distinctive_words(cluster_texts[cluster_id], n_words=10))
        all_cluster_words[cluster_id] = words

    overlaps = []
    unique_clusters = list(all_cluster_words.keys())
    for i, c1 in enumerate(unique_clusters):
        for c2 in unique_clusters[i+1:]:
            overlap = len(all_cluster_words[c1] & all_cluster_words[c2])
            overlaps.append(overlap)

    avg_word_overlap = np.mean(overlaps) if overlaps else 0
    print(f"\n   Average word overlap between clusters: {avg_word_overlap:.1f} words (out of 10)")
    print(f"   → {avg_word_overlap:.1f}: {'✓ Good' if avg_word_overlap < 2 else '⚠ Moderate' if avg_word_overlap < 4 else '✗ Too much overlap'}")

def compute_cluster_interpretability(metadata, cluster_labels):
    """Rate how easy it is to understand what each cluster is about."""
    print("\n" + "="*80)
    print("4. INTERPRETABILITY (can you name each cluster?)")
    print("="*80)

    cluster_texts = defaultdict(list)
    for headline, cluster_id in zip(metadata, cluster_labels):
        cluster_texts[cluster_id].append(headline['text'])

    print(f"\n   Sample headlines + topic interpretation:\n")

    for cluster_id in sorted(cluster_texts.keys()):
        headlines = cluster_texts[cluster_id][:2]
        print(f"   Cluster {cluster_id:2d}:")
        for h in headlines:
            print(f"      • {h[:70]}")
        print()

def compute_coverage_balance(cluster_labels):
    """Check if clusters are balanced or if one dominates."""
    print("\n" + "="*80)
    print("5. CLUSTER BALANCE (are clusters balanced in size?)")
    print("="*80)

    counts = Counter(cluster_labels)
    sizes = list(counts.values())

    print(f"\n   Cluster sizes:")
    for cluster_id in sorted(counts.keys()):
        size = counts[cluster_id]
        percentage = (size / len(cluster_labels)) * 100
        bar = "█" * int(percentage / 1) + "░" * (40 - int(percentage / 1))
        print(f"   Cluster {cluster_id:2d}: {bar} {percentage:5.1f}% ({size:4d})")

    imbalance = (max(sizes) - min(sizes)) / np.mean(sizes)
    print(f"\n   Imbalance ratio: {imbalance:.2f}")
    print(f"   → {imbalance:.2f}: {'✓ Balanced' if imbalance < 1.5 else '⚠ Moderate' if imbalance < 3.0 else '✗ Highly imbalanced'}")

def compute_overall_score(embeddings, cluster_labels, metadata):
    """Compute an overall quality score."""
    print("\n" + "="*80)
    print("OVERALL CLUSTERING QUALITY SCORE")
    print("="*80)

    # Get individual scores
    homogeneity = compute_semantic_homogeneity(embeddings, cluster_labels)
    avg_homogeneity = np.mean(list(homogeneity.values()))

    separation = compute_semantic_separation(embeddings, cluster_labels)

    # Normalize to 0-1 range
    homogeneity_score = min(avg_homogeneity, 1.0)  # Cap at 1
    separation_score = 1.0 - min(separation, 1.0)  # Invert (lower is better)

    # Combined score
    overall_score = (homogeneity_score + separation_score) / 2

    print(f"\n   Homogeneity score: {homogeneity_score:.3f}/1.0")
    print(f"   Separation score: {separation_score:.3f}/1.0")
    print(f"   OVERALL SCORE: {overall_score:.3f}/1.0")

    if overall_score > 0.7:
        print(f"\n   ✓ EXCELLENT clustering quality")
    elif overall_score > 0.5:
        print(f"\n   ✓ GOOD clustering quality")
    elif overall_score > 0.3:
        print(f"\n   ⚠ MODERATE clustering quality")
    else:
        print(f"\n   ✗ POOR clustering quality")

    return overall_score

print("Loading data...")
embeddings, metadata = load_data()

print("Clustering...")
cluster_labels = cluster_data(embeddings)

print("\nRunning text-aware clustering evaluation...\n")

# Run all evaluations
compute_semantic_homogeneity(embeddings, cluster_labels)
compute_semantic_separation(embeddings, cluster_labels)
compute_topic_distinctiveness(metadata, cluster_labels)
compute_cluster_interpretability(metadata, cluster_labels)
compute_coverage_balance(cluster_labels)
overall = compute_overall_score(embeddings, cluster_labels, metadata)

print("\n" + "="*80)
print("✅ TEXT-AWARE EVALUATION COMPLETE")
print("="*80)
print(f"\nThis eval focuses on what matters for topic clustering:")
print(f"  • Are headlines in same cluster similar? (homogeneity)")
print(f"  • Are headlines in different clusters different? (separation)")
print(f"  • Does each cluster have unique vocabulary? (distinctiveness)")
print(f"  • Can you interpret what each cluster is about? (interpretability)")
print(f"  • Are clusters balanced? (coverage)")


Loading data...
Clustering...

Running text-aware clustering evaluation...


1. SEMANTIC HOMOGENEITY (within-cluster similarity)
   Higher = headlines within cluster are similar to each other

   Cluster homogeneity scores:
   Cluster  5: ████████████████████ 1.000
   Cluster 17: ████████░░░░░░░░░░░░ 0.409
   Cluster  7: ███████░░░░░░░░░░░░░ 0.388
   Cluster  9: ███████░░░░░░░░░░░░░ 0.368
   Cluster 13: ███████░░░░░░░░░░░░░ 0.352
   Cluster  6: ██████░░░░░░░░░░░░░░ 0.345
   Cluster  0: ██████░░░░░░░░░░░░░░ 0.343
   Cluster 10: ██████░░░░░░░░░░░░░░ 0.330
   Cluster  3: ██████░░░░░░░░░░░░░░ 0.319
   Cluster 14: █████░░░░░░░░░░░░░░░ 0.299
   Cluster  2: █████░░░░░░░░░░░░░░░ 0.293
   Cluster  1: █████░░░░░░░░░░░░░░░ 0.287
   Cluster 12: ████░░░░░░░░░░░░░░░░ 0.240
   Cluster 15: ████░░░░░░░░░░░░░░░░ 0.230
   Cluster 18: ████░░░░░░░░░░░░░░░░ 0.216
   Cluster 19: ████░░░░░░░░░░░░░░░░ 0.202
   Cluster 16: ███░░░░░░░░░░░░░░░░░ 0.177
   Cluster 11: ██░░░░░░░░░░░░░░░░░░ 0.143
   Cluster  4: ██░░░

Loading data...
Clustering...

Running text-aware clustering evaluation...


================================================================================
1. SEMANTIC HOMOGENEITY (within-cluster similarity)
================================================================================
   Higher = headlines within cluster are similar to each other

   Cluster homogeneity scores:
   Cluster  5: ████████████████████ 1.000
   Cluster 17: ████████░░░░░░░░░░░░ 0.409
   Cluster  7: ███████░░░░░░░░░░░░░ 0.388
   Cluster  9: ███████░░░░░░░░░░░░░ 0.368
   Cluster 13: ███████░░░░░░░░░░░░░ 0.352
   Cluster  6: ██████░░░░░░░░░░░░░░ 0.345
   Cluster  0: ██████░░░░░░░░░░░░░░ 0.343
   Cluster 10: ██████░░░░░░░░░░░░░░ 0.330
   Cluster  3: ██████░░░░░░░░░░░░░░ 0.319
   Cluster 14: █████░░░░░░░░░░░░░░░ 0.299
   Cluster  2: █████░░░░░░░░░░░░░░░ 0.293
   Cluster  1: █████░░░░░░░░░░░░░░░ 0.287
   Cluster 12: ████░░░░░░░░░░░░░░░░ 0.240
   Cluster 15: ████░░░░░░░░░░░░░░░░ 0.230
   Cluster 18: ████░░░░░░░░░░░░░░░░ 0.216
   Cluster 19: ████░░░░░░░░░░░░░░░░ 0.202
   Cluster 16: ███░░░░░░░░░░░░░░░░░ 0.177
   Cluster 11: ██░░░░░░░░░░░░░░░░░░ 0.143
   Cluster  4: ██░░░░░░░░░░░░░░░░░░ 0.133
   Cluster  8: ██░░░░░░░░░░░░░░░░░░ 0.109

   Overall avg homogeneity: 0.309
   → 0.309: ✗ Needs improvement

================================================================================
2. SEMANTIC SEPARATION (between-cluster dissimilarity)
================================================================================
   Lower = clusters are distinct from each other

   Between-cluster similarity: 0.120
   (Headlines from different clusters have 0.120 avg similarity)
   → 0.120: ✓ Good separation

================================================================================
3. TOPIC DISTINCTIVENESS (unique vocabulary per cluster)
================================================================================
   Shows if clusters have unique, interpretable topics

   Top distinctive words per cluster:

   Cluster  0: israel, gaza, syria, iran, new
   Cluster  1: britain, british, labour, government, britons
   Cluster  2: europe, germany, france, new, eu
   Cluster  3: india, indians, pakistan, indian, modi
   Cluster  4: editor, letters, new, people, tiktok
   Cluster  5: markets, economic, data, commodities
   Cluster  6: trump, donald, america, tariffs, war
   Cluster  7: ukraine, russia, putin, europe, trump
   Cluster  8: cancer, work, world, health, new
   Cluster  9: ai, models, openai, world, new
   Cluster 10: africa, african, congo, sudan, war
   Cluster 11: world, new, war, life, people
   Cluster 12: america, maga, trump, democrats, american
   Cluster 13: china, chinese, economy, new, hong
   Cluster 14: investors, america, crypto, finance, market
   Cluster 15: president, milei, indonesia, colombia, brazil
   Cluster 16: climate, world, new, america, energy
   Cluster 17: china, america, taiwan, trump, trade
   Cluster 18: business, new, big, world, luxury
   Cluster 19: america, american, americans, big, pay

   Average word overlap between clusters: 1.4 words (out of 10)
   → 1.4: ✓ Good

================================================================================
4. INTERPRETABILITY (can you name each cluster?)
================================================================================

   Sample headlines + topic interpretation:

   Cluster  0:
      • Two months in, the Gaza ceasefire is floundering
      • Can Australia defend itself against jihadist murderers?

   Cluster  1:
      • How to heal the trauma from Northern Ireland’s killings
      • Britons are becoming obsessed with pet photography

   Cluster  2:
      • Europeans should be worried by America’s new national security strateg
      • Italy is using the Winter Olympics to appeal to the ultra-wealthy

   Cluster  3:
      • From honeycomb curry to blood fry: India’s “untouchable” cooking
      • How did one airline bring Indian aviation to its knees?

   Cluster  4:
      • Your Well Informed guide to surviving Christmas
      • How the young can make sense of the news

   Cluster  5:
      • Economic data, commodities and markets
      • Economic data, commodities and markets

   Cluster  6:
      • The tariffs that nearly stole Christmas
      • America’s Supreme Court should strike down Donald Trump’s tariffs

   Cluster  7:
      • Ukraine scrabbles for handholds against Russia’s massive assault
      • Two centuries ago, Russian revolutionaries tried to change the world

   Cluster  8:
      • How humankind’s 10m-year love affair with booze might end
      • How dogs make teens feel less anxious

   Cluster  9:
      • The race for an AI Jesus is on
      • How magicians stay relevant in the age of AI

   Cluster 10:
      • Ethiopia wants to build Africa’s biggest airport
      • Meet the American investors rushing into Congo

   Cluster 11:
      • Jimmy Lai’s judgment day
      • All sides have learned a lot from Extinction Rebellion’s co-founder

   Cluster 12:
      • A vote against gerrymandering shows why political courage is rare
      • More reasons for America’s friends to plan for the worst

   Cluster 13:
      • The Christmas-industrial complex centres on Yiwu
      • A journey along East Asia’s hidden artery

   Cluster 14:
      • The battle to stop clever people betting
      • Where America’s most prominent short-sellers are placing their bets

   Cluster 15:
      • The Economist’s country of the year for 2025
      • Javier Milei loosens his grip on the peso

   Cluster 16:
      • Will California try to block Hollywood’s next megadeal?
      • An oil boom where the Amazon meets the Atlantic

   Cluster 17:
      • China proved its strengths in 2025—and Donald Trump helped
      • Why has the border between the Koreas fallen silent?

   Cluster 18:
      • What Novo Nordisk, OpenAI and Pop Mart have in common
      • The plan to rescue Novo Nordisk

   Cluster 19:
      • Toll roads are spreading in America
      • More schools in America are adopting a four-day week


================================================================================
5. CLUSTER BALANCE (are clusters balanced in size?)
================================================================================

   Cluster sizes:
   Cluster  0: █████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   5.4% ( 198)
   Cluster  1: ███████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   7.6% ( 279)
   Cluster  2: █████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   5.6% ( 205)
   Cluster  3: ███░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   3.0% ( 110)
   Cluster  4: ██████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   6.4% ( 234)
   Cluster  5: █░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   1.4% (  51)
   Cluster  6: ███████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   7.8% ( 286)
   Cluster  7: ████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   4.7% ( 173)
   Cluster  8: ████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   4.9% ( 181)
   Cluster  9: ████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   4.1% ( 151)
   Cluster 10: ███░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   3.6% ( 132)
   Cluster 11: ██████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   6.6% ( 243)
   Cluster 12: █████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   5.6% ( 206)
   Cluster 13: █████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   5.5% ( 203)
   Cluster 14: ████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   4.7% ( 172)
   Cluster 15: ████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   4.8% ( 177)
   Cluster 16: ███░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   3.7% ( 135)
   Cluster 17: ████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   4.4% ( 162)
   Cluster 18: █████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   5.4% ( 198)
   Cluster 19: ████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   4.5% ( 165)

   Imbalance ratio: 1.28
   → 1.28: ✓ Balanced

================================================================================
OVERALL CLUSTERING QUALITY SCORE
================================================================================

================================================================================
1. SEMANTIC HOMOGENEITY (within-cluster similarity)
================================================================================
   Higher = headlines within cluster are similar to each other

   Cluster homogeneity scores:
   Cluster  5: ████████████████████ 1.000
   Cluster 17: ████████░░░░░░░░░░░░ 0.409
   Cluster  7: ███████░░░░░░░░░░░░░ 0.388
   Cluster  9: ███████░░░░░░░░░░░░░ 0.368
   Cluster 13: ███████░░░░░░░░░░░░░ 0.352
   Cluster  6: ██████░░░░░░░░░░░░░░ 0.345
   Cluster  0: ██████░░░░░░░░░░░░░░ 0.343
   Cluster 10: ██████░░░░░░░░░░░░░░ 0.330
   Cluster  3: ██████░░░░░░░░░░░░░░ 0.319
   Cluster 14: █████░░░░░░░░░░░░░░░ 0.299
   Cluster  2: █████░░░░░░░░░░░░░░░ 0.293
   Cluster  1: █████░░░░░░░░░░░░░░░ 0.287
   Cluster 12: ████░░░░░░░░░░░░░░░░ 0.240
   Cluster 15: ████░░░░░░░░░░░░░░░░ 0.230
   Cluster 18: ████░░░░░░░░░░░░░░░░ 0.216
   Cluster 19: ████░░░░░░░░░░░░░░░░ 0.202
   Cluster 16: ███░░░░░░░░░░░░░░░░░ 0.177
   Cluster 11: ██░░░░░░░░░░░░░░░░░░ 0.143
   Cluster  4: ██░░░░░░░░░░░░░░░░░░ 0.133
   Cluster  8: ██░░░░░░░░░░░░░░░░░░ 0.109

   Overall avg homogeneity: 0.309
   → 0.309: ✗ Needs improvement

================================================================================
2. SEMANTIC SEPARATION (between-cluster dissimilarity)
================================================================================
   Lower = clusters are distinct from each other

   Between-cluster similarity: 0.121
   (Headlines from different clusters have 0.121 avg similarity)
   → 0.121: ✓ Good separation

   Homogeneity score: 0.309/1.0
   Separation score: 0.879/1.0
   OVERALL SCORE: 0.594/1.0

   ✓ GOOD clustering quality

================================================================================
✅ TEXT-AWARE EVALUATION COMPLETE
================================================================================

This eval focuses on what matters for topic clustering:
  • Are headlines in same cluster similar? (homogeneity)
  • Are headlines in different clusters different? (separation)
  • Does each cluster have unique vocabulary? (distinctiveness)
  • Can you interpret what each cluster is about? (interpretability)
  • Are clusters balanced? (coverage)

In the future


*   Including more than headline. Maybe first few graphs
*   argmax # clusters for homogeneity
* Further validate with other magazines

